In [11]:
import os
import sys
import json
sys.path.append('/home/gpsnest/anaconda3/envs/qa-eval/lib/python3.6/site-packages')

In [2]:
from elasticsearch import Elasticsearch

config = {'host':'localhost', 'port':9200}
es = Elasticsearch([config])

# test connection
es.ping()

True

In [47]:
# create new index
index_config = {
    "settings": {
        "analysis": {
            "analyzer": {
                "stop_stem_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter":[
                        "lowercase",
                        "stop",
                        "snowball"
                    ]
                    
                }
            }
        }
    },
    "mappings": {
        "dynamic": "strict", 
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text", "analyzer": "stop_stem_analyzer"},
            "text": {"type": "text", "analyzer": "stop_stem_analyzer"},
            "url": {"type": "text"}
            }
        }
    }


In [48]:
index_name = 'wiki-full-stop-stem'
es.indices.create(index=index_name, body=index_config, ignore=400)

{'acknowledged': True,
 'index': 'wiki-full-stop-stem',
 'shards_acknowledged': True}

In [46]:
# es.indices.delete(index=index_name)

{'acknowledged': True}

In [24]:
data = []
with open('../data/AA/wiki_00', 'rb') as f:
    for line in f:
        data.append(json.loads(line))

In [30]:
data[0].keys()

dict_keys(['id', 'url', 'title', 'text'])

In [ ]:
''

In [49]:
# collapse-hide
from tqdm import tqdm
from time import sleep
import os

def populate_wiki_index(es_obj, index_name, data_dir):
    '''
    Loads records into an existing Elasticsearch index from Wikiextractor JSON dump

    Args:
        es_obj (elasticsearch.client.Elasticsearch) - Elasticsearch client object
        index_name (str) - Name of index
        evidence_corpus (list) - List of dicts containing data records

    '''
    
    files = []
    for dirname, dirnames, filenames in os.walk(data_dir):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    
    
    for file in tqdm(files):
        
        data = []
        with open(file, 'rb') as f:
            for line in f:
                data.append(json.loads(line))

        for i, rec in enumerate(data):
            try:
                index_status = es_obj.index(index=index_name, body=rec)
            except:
                print(f'Unable to load document: {file, i, rec["title"]}.')
            
    sleep(5)
    n_records = es_obj.count(index=index_name)['count']
    print(f'Succesfully loaded {n_records} into {index_name}')


    return

In [50]:
%%time

data_dir = '/home/gpsnest/Documents/andrew/neuralqa-eval/data'
populate_wiki_index(es_obj=es, index_name='wiki-full-stop-stem', data_dir=data_dir)

100%|██████████| 13969/13969 [23:27:53<00:00,  6.05s/it]   


Succesfully loaded 6105228 into wiki-full-stop-stem
CPU times: user 2h 28min 1s, sys: 11min 32s, total: 2h 39min 33s
Wall time: 23h 28min 1s


In [52]:
os.system("pwd")

0

In [55]:
os.path.exists('../data')

True